In [1]:
import os
import sys
import pandas as pd
import numpy as np
pd.options.display.max_columns=200

In [2]:

sys.path

['/Users/hiroshi/milize_project/python/udemy/Soledad Galli/deploying-machine-learning-models/jupyter_notebooks',
 '/Users/hiroshi/.pyenv/versions/anaconda3-2019.03/lib/python37.zip',
 '/Users/hiroshi/.pyenv/versions/anaconda3-2019.03/lib/python3.7',
 '/Users/hiroshi/.pyenv/versions/anaconda3-2019.03/lib/python3.7/lib-dynload',
 '',
 '/Users/hiroshi/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages',
 '/Users/hiroshi/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/aeosa',
 '/Users/hiroshi/milize_project/python/udemy/Soledad Galli/deploying-machine-learning-models/packages/regression_model',
 '/Users/hiroshi/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/IPython/extensions',
 '/Users/hiroshi/.ipython']

In [3]:
from regression_model.processing.data_management import  load_dataset

In [4]:
from regression_model.config import config

In [5]:

data = load_dataset(file_name='train.csv')

In [6]:


data


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,...,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,...,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,...,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,...,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,...,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,...,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,...,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,...,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,...,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,...,4,2010,WD,Normal,142125


In [7]:
config.FEATURES

['MSSubClass',
 'MSZoning',
 'Neighborhood',
 'OverallQual',
 'OverallCond',
 'YearRemodAdd',
 'RoofStyle',
 'MasVnrType',
 'BsmtQual',
 'BsmtExposure',
 'HeatingQC',
 'CentralAir',
 '1stFlrSF',
 'GrLivArea',
 'BsmtFullBath',
 'KitchenQual',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageCars',
 'PavedDrive',
 'LotFrontage',
 'YrSold']

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        data[config.FEATURES],
        data[config.TARGET],
        test_size=0.1,
        random_state=0)  # we are setting the seed here

In [9]:
# transform the target
y_train = np.log(y_train)
y_test = np.log(y_test)

# CategoricalImputerについて

In [10]:
config.CATEGORICAL_VARS_WITH_NA

['MasVnrType',
 'BsmtQual',
 'BsmtExposure',
 'FireplaceQu',
 'GarageType',
 'GarageFinish']

In [11]:
X_train[config.CATEGORICAL_VARS_WITH_NA]

,MasVnrType,BsmtQual,BsmtExposure,FireplaceQu,GarageType,GarageFinish
930,None,Gd,Av,NaN,Attchd,Fin
656,BrkFace,TA,No,NaN,Attchd,RFn
45,BrkFace,Ex,No,Gd,Attchd,RFn
1348,None,Gd,Gd,Fa,Attchd,RFn
55,BrkFace,TA,No,Gd,Attchd,RFn
...,...,...,...,...,...,...
763,BrkFace,Gd,Mn,Gd,Attchd,RFn
835,None,Gd,No,NaN,Attchd,Unf
1216,None,NaN,NaN,NaN,Attchd,Unf
559,BrkFace,Gd,Gd,TA,Attchd,Fin


In [12]:
from regression_model.processing import preprocessors as pp

model_categoricalImputer = pp.CategoricalImputer(variables=config.CATEGORICAL_VARS_WITH_NA)


In [13]:
model_categoricalImputer.fit(X_train, y_train)

CategoricalImputer(variables=['MasVnrType', 'BsmtQual', 'BsmtExposure', 'FireplaceQu', 'GarageType', 'GarageFinish'])

In [14]:
# missingに埋められていることを確認

model_categoricalImputer.transform(X_train) 

,MSSubClass,MSZoning,Neighborhood,OverallQual,OverallCond,...,GarageFinish,GarageCars,PavedDrive,LotFrontage,YrSold
930,20,RL,Timber,8,5,...,Fin,3,Y,73.0,2009
656,20,RL,NAmes,5,7,...,RFn,1,Y,72.0,2008
45,120,RL,NridgHt,9,5,...,RFn,2,Y,61.0,2010
1348,20,RL,SawyerW,7,5,...,RFn,2,Y,NaN,2007
55,20,RL,NAmes,6,5,...,RFn,2,Y,100.0,2008
...,...,...,...,...,...,...,...,...,...,...,...
763,60,RL,NoRidge,8,5,...,RFn,3,Y,82.0,2009
835,20,RL,Sawyer,4,7,...,Unf,2,Y,60.0,2010
1216,90,RM,Sawyer,6,5,...,Unf,2,Y,68.0,2010
559,120,RL,Blmngtn,7,5,...,Fin,2,Y,NaN,2006


In [15]:
model_categoricalImputer.fit_transform(X_train) == model_categoricalImputer.transform(X_train) 

,MSSubClass,MSZoning,Neighborhood,OverallQual,OverallCond,...,GarageFinish,GarageCars,PavedDrive,LotFrontage,YrSold
930,True,True,True,True,True,...,True,True,True,True,True
656,True,True,True,True,True,...,True,True,True,True,True
45,True,True,True,True,True,...,True,True,True,True,True
1348,True,True,True,True,True,...,True,True,True,False,True
55,True,True,True,True,True,...,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
763,True,True,True,True,True,...,True,True,True,True,True
835,True,True,True,True,True,...,True,True,True,True,True
1216,True,True,True,True,True,...,True,True,True,True,True
559,True,True,True,True,True,...,True,True,True,False,True


# NumericalImputerについて
* 最頻出の値を入れる#

In [16]:
model_numericalImputer = pp.NumericalImputer(variables=config.NUMERICAL_VARS_WITH_NA)



In [17]:
# LotFrontage: 間口の広さ
config.NUMERICAL_VARS_WITH_NA

['LotFrontage']

In [18]:
#model_numericalImputer.fit(X_train,y_train)

In [19]:
X_train['LotFrontage']

930      73.0
656      72.0
45       61.0
1348      NaN
55      100.0
        ...  
763      82.0
835      60.0
1216     68.0
559       NaN
684      58.0
Name: LotFrontage, Length: 1314, dtype: float64

In [20]:
# エラーになる

model_numericalImputer.transform(X_train) ['LotFrontage']

AttributeError: 'NumericalImputer' object has no attribute 'imputer_dict_'

In [21]:
# fit_transformはfit + transform
model_numericalImputer.fit_transform(X_train) ['LotFrontage']

930      73.0
656      72.0
45       61.0
1348     60.0
55      100.0
        ...  
763      82.0
835      60.0
1216     68.0
559      60.0
684      58.0
Name: LotFrontage, Length: 1314, dtype: float64

In [22]:
model_numericalImputer.transform(X_train) ['LotFrontage']


930      73.0
656      72.0
45       61.0
1348     60.0
55      100.0
        ...  
763      82.0
835      60.0
1216     68.0
559      60.0
684      58.0
Name: LotFrontage, Length: 1314, dtype: float64

In [23]:
# pandas.Seriesおよびpandas.DataFrameのメソッドmode()を使うと、
# 各列・各行の最頻値を取得できる。
X_train['LotFrontage'].mode()

0    60.0
dtype: float64

In [24]:
# 確かに一番多いことがわかる
X_train['LotFrontage'].value_counts().head()

60.0    127
80.0     63
70.0     60
50.0     50
75.0     46
Name: LotFrontage, dtype: int64

# TemporalVariableEstimatorについて


In [25]:
# YearRemodAdd

X_train[config.TEMPORAL_VARS]

930     2007
656     2006
45      2005
1348    1998
55      1964
        ... 
763     1999
835     1995
1216    1978
559     2004
684     1998
Name: YearRemodAdd, Length: 1314, dtype: int64

In [26]:
# YearRemodAdd : Remodel date
config.TEMPORAL_VARS

'YearRemodAdd'

In [27]:
# YrSold: Year Sold
config.DROP_FEATURES

'YrSold'

In [28]:
# variablesをreference_variableから引くという手順
# 今回は売れた年 - リノベした年
model_temporalVariableEstimator =pp.TemporalVariableEstimator(
                variables=config.TEMPORAL_VARS,
                reference_variable=config.DROP_FEATURES)




In [29]:
model_temporalVariableEstimator.transform(X_train)[config.TEMPORAL_VARS]

930      2
656      2
45       5
1348     9
55      44
        ..
763     10
835     15
1216    32
559      2
684     12
Name: YearRemodAdd, Length: 1314, dtype: int64

# RareLabelCategoricalEncoderについて



In [30]:
config.CATEGORICAL_VARS

['MSZoning',
 'Neighborhood',
 'RoofStyle',
 'MasVnrType',
 'BsmtQual',
 'BsmtExposure',
 'HeatingQC',
 'CentralAir',
 'KitchenQual',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'PavedDrive']

In [31]:
X_train[config.CATEGORICAL_VARS]

,MSZoning,Neighborhood,RoofStyle,MasVnrType,BsmtQual,...,KitchenQual,FireplaceQu,GarageType,GarageFinish,PavedDrive
930,RL,Timber,Gable,None,Gd,...,Gd,NaN,Attchd,Fin,Y
656,RL,NAmes,Gable,BrkFace,TA,...,Gd,NaN,Attchd,RFn,Y
45,RL,NridgHt,Hip,BrkFace,Ex,...,Ex,Gd,Attchd,RFn,Y
1348,RL,SawyerW,Gable,None,Gd,...,Gd,Fa,Attchd,RFn,Y
55,RL,NAmes,Gable,BrkFace,TA,...,TA,Gd,Attchd,RFn,Y
...,...,...,...,...,...,...,...,...,...,...,...
763,RL,NoRidge,Gable,BrkFace,Gd,...,Gd,Gd,Attchd,RFn,Y
835,RL,Sawyer,Gable,None,Gd,...,Gd,NaN,Attchd,Unf,Y
1216,RM,Sawyer,Gable,None,NaN,...,TA,NaN,Attchd,Unf,Y
559,RL,Blmngtn,Gable,BrkFace,Gd,...,Gd,TA,Attchd,Fin,Y


In [32]:
model_RareLabelCategoricalEncoder = pp.RareLabelCategoricalEncoder(
                tol=0.01,
                variables=config.CATEGORICAL_VARS)


In [33]:
model_RareLabelCategoricalEncoder

RareLabelCategoricalEncoder(tol=0.01,
              variables=['MSZoning', 'Neighborhood', 'RoofStyle', 'MasVnrType', 'BsmtQual', 'BsmtExposure', 'HeatingQC', 'CentralAir', 'KitchenQual', 'FireplaceQu', 'GarageType', 'GarageFinish', 'PavedDrive'])

In [34]:
model_RareLabelCategoricalEncoder.fit_transform(X_train)

,MSSubClass,MSZoning,Neighborhood,OverallQual,OverallCond,...,GarageFinish,GarageCars,PavedDrive,LotFrontage,YrSold
930,20,RL,Timber,8,5,...,Fin,3,Y,73.0,2009
656,20,RL,NAmes,5,7,...,RFn,1,Y,72.0,2008
45,120,RL,NridgHt,9,5,...,RFn,2,Y,61.0,2010
1348,20,RL,SawyerW,7,5,...,RFn,2,Y,NaN,2007
55,20,RL,NAmes,6,5,...,RFn,2,Y,100.0,2008
...,...,...,...,...,...,...,...,...,...,...,...
763,60,RL,NoRidge,8,5,...,RFn,3,Y,82.0,2009
835,20,RL,Sawyer,4,7,...,Unf,2,Y,60.0,2010
1216,90,RM,Sawyer,6,5,...,Unf,2,Y,68.0,2010
559,120,RL,Blmngtn,7,5,...,Fin,2,Y,NaN,2006


In [35]:
model_RareLabelCategoricalEncoder.encoder_dict_

{'MSZoning': ['RL', 'RM', 'FV', 'RH'],
 'Neighborhood': ['NAmes',
  'CollgCr',
  'OldTown',
  'Edwards',
  'Somerst',
  'NridgHt',
  'Gilbert',
  'Sawyer',
  'NWAmes',
  'BrkSide',
  'SawyerW',
  'Crawfor',
  'Mitchel',
  'Timber',
  'NoRidge',
  'IDOTRR',
  'ClearCr',
  'SWISU',
  'StoneBr',
  'Blmngtn',
  'MeadowV',
  'BrDale'],
 'RoofStyle': ['Gable', 'Hip'],
 'MasVnrType': ['None', 'BrkFace', 'Stone'],
 'BsmtQual': ['TA', 'Gd', 'Ex', 'Fa'],
 'BsmtExposure': ['No', 'Av', 'Gd', 'Mn'],
 'HeatingQC': ['Ex', 'TA', 'Gd', 'Fa'],
 'CentralAir': ['Y', 'N'],
 'KitchenQual': ['TA', 'Gd', 'Ex', 'Fa'],
 'FireplaceQu': ['Gd', 'TA', 'Fa', 'Ex', 'Po'],
 'GarageType': ['Attchd', 'Detchd', 'BuiltIn', 'Basment'],
 'GarageFinish': ['Unf', 'RFn', 'Fin'],
 'PavedDrive': ['Y', 'N', 'P']}

In [36]:
encoder_dict_ = {}

for var in config.CATEGORICAL_VARS:
    # the encoder will learn the most frequent categories
    t = pd.Series(X_train[var].value_counts() / np.float(len(X_train)))
    # frequent labels:
    encoder_dict_[var] = list(t[t >= 0.01].index)
    break


In [37]:
encoder_dict_

{'MSZoning': ['RL', 'RM', 'FV', 'RH']}

In [38]:
t

RL         0.788432
RM         0.149924
FV         0.041857
RH         0.012177
C (all)    0.007610
Name: MSZoning, dtype: float64

In [39]:
var

'MSZoning'

In [40]:
X_train[var]

930     RL
656     RL
45      RL
1348    RL
55      RL
        ..
763     RL
835     RL
1216    RM
559     RL
684     RL
Name: MSZoning, Length: 1314, dtype: object

In [41]:
X_train[var].value_counts()

RL         1036
RM          197
FV           55
RH           16
C (all)      10
Name: MSZoning, dtype: int64

In [42]:
np.float(len(X_train))

1314.0

In [43]:
# fit : 0.01（1%以上のカテゴリかるデータを抽出）
# transform : 1%以上のカテゴを「Rare」というデータに置換


# CategoricalEncoderについて

In [64]:
config.CATEGORICAL_VARS

['MSZoning',
 'Neighborhood',
 'RoofStyle',
 'MasVnrType',
 'BsmtQual',
 'BsmtExposure',
 'HeatingQC',
 'CentralAir',
 'KitchenQual',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'PavedDrive']

In [65]:
X_train[config.CATEGORICAL_VARS]

,MSZoning,Neighborhood,RoofStyle,MasVnrType,BsmtQual,...,KitchenQual,FireplaceQu,GarageType,GarageFinish,PavedDrive
930,RL,Timber,Gable,None,Gd,...,Gd,NaN,Attchd,Fin,Y
656,RL,NAmes,Gable,BrkFace,TA,...,Gd,NaN,Attchd,RFn,Y
45,RL,NridgHt,Hip,BrkFace,Ex,...,Ex,Gd,Attchd,RFn,Y
1348,RL,SawyerW,Gable,None,Gd,...,Gd,Fa,Attchd,RFn,Y
55,RL,NAmes,Gable,BrkFace,TA,...,TA,Gd,Attchd,RFn,Y
...,...,...,...,...,...,...,...,...,...,...,...
763,RL,NoRidge,Gable,BrkFace,Gd,...,Gd,Gd,Attchd,RFn,Y
835,RL,Sawyer,Gable,None,Gd,...,Gd,NaN,Attchd,Unf,Y
1216,RM,Sawyer,Gable,None,NaN,...,TA,NaN,Attchd,Unf,Y
559,RL,Blmngtn,Gable,BrkFace,Gd,...,Gd,TA,Attchd,Fin,Y


In [46]:
model_CategoricalEncoder = pp.CategoricalEncoder(variables=config.CATEGORICAL_VARS)



In [47]:
model_CategoricalEncoder.fit(X_train, y_train)

CategoricalEncoder(variables=['MSZoning', 'Neighborhood', 'RoofStyle', 'MasVnrType', 'BsmtQual', 'BsmtExposure', 'HeatingQC', 'CentralAir', 'KitchenQual', 'FireplaceQu', 'GarageType', 'GarageFinish', 'PavedDrive'])

In [48]:


X_train_RareLabelCategoricalEncoder = model_RareLabelCategoricalEncoder.fit_transform(X_train)



In [49]:
X_train_RareLabelCategoricalEncoder

,MSSubClass,MSZoning,Neighborhood,OverallQual,OverallCond,...,GarageFinish,GarageCars,PavedDrive,LotFrontage,YrSold
930,20,RL,Timber,8,5,...,Fin,3,Y,73.0,2009
656,20,RL,NAmes,5,7,...,RFn,1,Y,72.0,2008
45,120,RL,NridgHt,9,5,...,RFn,2,Y,61.0,2010
1348,20,RL,SawyerW,7,5,...,RFn,2,Y,NaN,2007
55,20,RL,NAmes,6,5,...,RFn,2,Y,100.0,2008
...,...,...,...,...,...,...,...,...,...,...,...
763,60,RL,NoRidge,8,5,...,RFn,3,Y,82.0,2009
835,20,RL,Sawyer,4,7,...,Unf,2,Y,60.0,2010
1216,90,RM,Sawyer,6,5,...,Unf,2,Y,68.0,2010
559,120,RL,Blmngtn,7,5,...,Fin,2,Y,NaN,2006


# CategoricalEncoder

In [50]:
temp = pd.concat([X_train_RareLabelCategoricalEncoder
                  , y_train], axis=1)
temp.columns = list(X_train_RareLabelCategoricalEncoder.columns) + ['target']

# persist transforming dictionary
encoder_dict_ = {}

for var in config.CATEGORICAL_VARS:
    t = temp.groupby([var])['target'].mean().sort_values(
        ascending=True).index
    encoder_dict_[var] = {k: i for i, k in enumerate(t, 0)}
    
    

In [51]:
 temp.groupby([var])['target'].mean()

PavedDrive
N    11.571606
P    11.770259
Y    12.062518
Name: target, dtype: float64

In [52]:
t

Index(['N', 'P', 'Y'], dtype='object', name='PavedDrive')

In [53]:
temp

,MSSubClass,MSZoning,Neighborhood,OverallQual,OverallCond,...,GarageCars,PavedDrive,LotFrontage,YrSold,target
930,20,RL,Timber,8,5,...,3,Y,73.0,2009,12.211060
656,20,RL,NAmes,5,7,...,1,Y,72.0,2008,11.887931
45,120,RL,NridgHt,9,5,...,2,Y,61.0,2010,12.675764
1348,20,RL,SawyerW,7,5,...,2,Y,NaN,2007,12.278393
55,20,RL,NAmes,6,5,...,2,Y,100.0,2008,12.103486
...,...,...,...,...,...,...,...,...,...,...,...
763,60,RL,NoRidge,8,5,...,3,Y,82.0,2009,12.727838
835,20,RL,Sawyer,4,7,...,2,Y,60.0,2010,11.759786
1216,90,RM,Sawyer,6,5,...,2,Y,68.0,2010,11.626254
559,120,RL,Blmngtn,7,5,...,2,Y,NaN,2006,12.363076


In [54]:
encoder_dict_

{'MSZoning': {'Rare': 0, 'RM': 1, 'RH': 2, 'RL': 3, 'FV': 4},
 'Neighborhood': {'IDOTRR': 0,
  'MeadowV': 1,
  'BrDale': 2,
  'Edwards': 3,
  'BrkSide': 4,
  'OldTown': 5,
  'Sawyer': 6,
  'SWISU': 7,
  'NAmes': 8,
  'Mitchel': 9,
  'SawyerW': 10,
  'Rare': 11,
  'NWAmes': 12,
  'Gilbert': 13,
  'Blmngtn': 14,
  'CollgCr': 15,
  'Crawfor': 16,
  'ClearCr': 17,
  'Somerst': 18,
  'Timber': 19,
  'StoneBr': 20,
  'NridgHt': 21,
  'NoRidge': 22},
 'RoofStyle': {'Gable': 0, 'Rare': 1, 'Hip': 2},
 'MasVnrType': {'None': 0, 'Rare': 1, 'BrkFace': 2, 'Stone': 3},
 'BsmtQual': {'Rare': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4},
 'BsmtExposure': {'Rare': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4},
 'HeatingQC': {'Rare': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4},
 'CentralAir': {'N': 0, 'Y': 1},
 'KitchenQual': {'Fa': 0, 'TA': 1, 'Gd': 2, 'Ex': 3},
 'FireplaceQu': {'Po': 0, 'Rare': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
 'GarageType': {'Rare': 0,
  'Detchd': 1,
  'Basment': 2,
  'Attchd': 3,
  'BuiltIn': 

In [55]:
X_train_RareLabelCategoricalEncoder['MSZoning']

930     RL
656     RL
45      RL
1348    RL
55      RL
        ..
763     RL
835     RL
1216    RM
559     RL
684     RL
Name: MSZoning, Length: 1314, dtype: object

In [66]:
X_train[config.CATEGORICAL_VARS]

,MSZoning,Neighborhood,RoofStyle,MasVnrType,BsmtQual,...,KitchenQual,FireplaceQu,GarageType,GarageFinish,PavedDrive
930,RL,Timber,Gable,None,Gd,...,Gd,NaN,Attchd,Fin,Y
656,RL,NAmes,Gable,BrkFace,TA,...,Gd,NaN,Attchd,RFn,Y
45,RL,NridgHt,Hip,BrkFace,Ex,...,Ex,Gd,Attchd,RFn,Y
1348,RL,SawyerW,Gable,None,Gd,...,Gd,Fa,Attchd,RFn,Y
55,RL,NAmes,Gable,BrkFace,TA,...,TA,Gd,Attchd,RFn,Y
...,...,...,...,...,...,...,...,...,...,...,...
763,RL,NoRidge,Gable,BrkFace,Gd,...,Gd,Gd,Attchd,RFn,Y
835,RL,Sawyer,Gable,None,Gd,...,Gd,NaN,Attchd,Unf,Y
1216,RM,Sawyer,Gable,None,NaN,...,TA,NaN,Attchd,Unf,Y
559,RL,Blmngtn,Gable,BrkFace,Gd,...,Gd,TA,Attchd,Fin,Y


In [72]:
X = X_train.copy()
for feature in config.CATEGORICAL_VARS:
    X[feature] = X[feature].map(encoder_dict_[feature])


In [73]:
X

,MSSubClass,MSZoning,Neighborhood,OverallQual,OverallCond,...,GarageFinish,GarageCars,PavedDrive,LotFrontage,YrSold
930,20,3.0,19.0,8,5,...,3.0,3,2,73.0,2009
656,20,3.0,8.0,5,7,...,2.0,1,2,72.0,2008
45,120,3.0,21.0,9,5,...,2.0,2,2,61.0,2010
1348,20,3.0,10.0,7,5,...,2.0,2,2,NaN,2007
55,20,3.0,8.0,6,5,...,2.0,2,2,100.0,2008
...,...,...,...,...,...,...,...,...,...,...,...
763,60,3.0,22.0,8,5,...,2.0,3,2,82.0,2009
835,20,3.0,6.0,4,7,...,1.0,2,2,60.0,2010
1216,90,1.0,6.0,6,5,...,1.0,2,2,68.0,2010
559,120,3.0,14.0,7,5,...,3.0,2,2,NaN,2006


In [74]:
# X = X.fillna(0)

In [75]:
# check if transformer introduces NaN
from regression_model.processing.errors import InvalidModelInputError

if X[config.CATEGORICAL_VARS].isnull().any().any():
    null_counts = X[config.CATEGORICAL_VARS].isnull().any()
    vars_ = {key: value for (key, value) in null_counts.items()
             if value is True}
    raise InvalidModelInputError(
        f'Categorical encoder has introduced NaN when '
        f'transforming categorical variables: {vars_.keys()}')


InvalidModelInputError: Categorical encoder has introduced NaN when transforming categorical variables: dict_keys(['MSZoning', 'Neighborhood', 'RoofStyle', 'MasVnrType', 'BsmtQual', 'BsmtExposure', 'HeatingQC', 'FireplaceQu', 'GarageType', 'GarageFinish'])

In [71]:
X[config.CATEGORICAL_VARS].isnull()

,MSZoning,Neighborhood,RoofStyle,MasVnrType,BsmtQual,...,KitchenQual,FireplaceQu,GarageType,GarageFinish,PavedDrive
930,False,False,False,False,False,...,False,False,False,False,False
656,False,False,False,False,False,...,False,False,False,False,False
45,False,False,False,False,False,...,False,False,False,False,False
1348,False,False,False,False,False,...,False,False,False,False,False
55,False,False,False,False,False,...,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
763,False,False,False,False,False,...,False,False,False,False,False
835,False,False,False,False,False,...,False,False,False,False,False
1216,False,False,False,False,False,...,False,False,False,False,False
559,False,False,False,False,False,...,False,False,False,False,False


In [76]:
X[config.CATEGORICAL_VARS].isnull().any()

MSZoning         True
Neighborhood     True
RoofStyle        True
MasVnrType       True
BsmtQual         True
                ...  
KitchenQual     False
FireplaceQu      True
GarageType       True
GarageFinish     True
PavedDrive      False
Length: 13, dtype: bool

In [62]:
X[config.CATEGORICAL_VARS].isnull().any().any()

True

In [77]:
X[config.CATEGORICAL_VARS].isnull().any()

MSZoning         True
Neighborhood     True
RoofStyle        True
MasVnrType       True
BsmtQual         True
                ...  
KitchenQual     False
FireplaceQu      True
GarageType       True
GarageFinish     True
PavedDrive      False
Length: 13, dtype: bool

In [78]:
{key: value for (key, value) in null_counts.items()
             if value is True}

{'MSZoning': True,
 'Neighborhood': True,
 'RoofStyle': True,
 'MasVnrType': True,
 'BsmtQual': True,
 'BsmtExposure': True,
 'HeatingQC': True,
 'FireplaceQu': True,
 'GarageType': True,
 'GarageFinish': True}

In [79]:
vars_ = {}
for (key, value) in null_counts.items():
    if value is True:
        vars_[key] = value
    

In [80]:
vars_

{'MSZoning': True,
 'Neighborhood': True,
 'RoofStyle': True,
 'MasVnrType': True,
 'BsmtQual': True,
 'BsmtExposure': True,
 'HeatingQC': True,
 'FireplaceQu': True,
 'GarageType': True,
 'GarageFinish': True}